In [1]:
import copy
import datetime
import isois
import numpy as np
import spacepy.pycdf
import types

import data_jat as data


This unreleased version of SpacePy is not supported by the SpacePy team.


In [2]:
d = data.Data('joyce-apj-tab2')
d.read_data()


Found cached data from file eventdata_joyce-apj-tab2_20210420.pickle3.bz2, using contents...


In [3]:
# check eventtimes
times = np.array([('2018-296T14:00:00', '2018-314T16:00:00'),
                  ('2018-325T08:00:00', '2018-327T00:00:00'),
                  ('2019-007T00:00:00', '2019-011T12:00:00'),
                  ('2019-181T09:00:00', '2019-191T18:00:00')])
string2datetime = np.vectorize(lambda s: datetime.datetime.strptime(s, '%Y-%jT%H:%M:%S')) # probably unnecessary, but...
times = string2datetime(times)

assert(all(times[0] == d.eventtimes[0]))
assert(all(times[1] == d.eventtimes[1]))
assert(all(times[2] == d.eventtimes[2]))
assert(all(times[-1] == d.eventtimes[-1]))

print('This printed, so apparently event/quiet time handling is working!')


This printed, so apparently event/quiet time handling is working!


In [4]:
# check data contents
e0 = d.eventdata[0]['ChanP']['epoch'][0]
eF = d.eventdata[0]['ChanP']['epoch'][-1]
files = d._get_files('psp_isois-epilo_l2-ic',
                     spacepy.pycdf.lib.tt2000_to_datetime(e0),
                     spacepy.pycdf.lib.tt2000_to_datetime(eF))

c = spacepy.pycdf.CDF(files[0])
f_start = np.searchsorted(c.raw_var('Epoch_ChanP'), e0)
read_to = len(c['Epoch_ChanP']) - f_start

assert(np.all(c.raw_var('Epoch_ChanP')[f_start:f_start + read_to] == d.eventdata[0]['ChanP']['epoch'][:read_to]))
nonnan = ~np.isnan(d.eventdata[0]['ChanP']['flux'][:read_to])
assert(np.all((c['H_Flux_ChanP'][f_start:f_start + read_to] == d.eventdata[0]['ChanP']['flux'][:read_to])[nonnan]))

print('This printed, so apparently file creation is working!')


This printed, so apparently file creation is working!


In [5]:
# check that energy_unc_plus and energy_unc_minus are the same everywhere
for i, f in enumerate(d.eventdata):
    e = types.SimpleNamespace(**copy.deepcopy(f['ChanP']))
    
    nonnan_plus = ~np.isnan(e.energy_unc_plus)
    nonnan_minus = ~np.isnan(e.energy_unc_minus)
    assert(np.all(nonnan_plus == nonnan_minus))

    assert(np.all((e.energy_unc_plus[0, 0, :] == e.energy_unc_plus)[nonnan_plus]))
    assert(np.all((e.energy_unc_minus[0, 0, :] == e.energy_unc_minus)[nonnan_minus]))

print('This printed, so apparently all the energy deltas are consistent across look direction!')


This printed, so apparently all the energy deltas are consistent across look direction!
